In [ ]:
import torch
from models import get_pretrained_pretransform
from export import remove_parametrizations

import librosa, time
from IPython.display import Audio, display

import cached_conv as cc

cc.use_cached_conv(True)

device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
## Load the autoencoder from stable-audio-open-1.0

autoencoder, model_config = get_pretrained_pretransform("stabilityai/stable-audio-open-1.0", 
                                                         model_half=True, 
                                                         skip_bottleneck=True, 
                                                         device=device)

print(f"sample_rate: {model_config.get('sample_rate', 'unknown')}")
print(f"latent_dim: {model_config['model']['pretransform']['config'].get('latent_dim', 'unknown')}")
print(f"downsampling_ratio: {model_config['model']['pretransform']['config'].get('downsampling_ratio', 'unknown')}")
print(f"io_channels: {model_config['model']['pretransform']['config'].get('io_channels', 'unknown')}")

autoencoder = autoencoder.to(device)
autoencoder.eval()

remove_parametrizations(autoencoder)

In [ ]:
# Load an example audio file and split into chunks

buffer_size = 4096

audio_path = librosa.example('fishin', hq=True)
wv, sr = librosa.load(audio_path, sr=44100, mono=False)
wv = torch.tensor(wv, device=device)[:,buffer_size*50:buffer_size*150].unsqueeze(0)  # make stereo, limit length for test
wv_chunks = [wv[:, :, i*buffer_size:(i+1)*buffer_size] for i in range(100)]

print(f'waveform shape: {wv.shape}')
print(f'number of chunks: {len(wv_chunks)}')
print(f'chunk shape: {wv_chunks[0].shape}')

In [ ]:
print(f'Running encoder, test device: {device}')
## Run audio chunks to the encoder

latent_chunks = []
with torch.no_grad():
    torch.cuda.synchronize() if device == "cuda" else torch.mps.synchronize()
    start_time = time.perf_counter()
    for i, w in enumerate(wv_chunks):
        latent = autoencoder.encode(w)
        latent_chunks.append(latent)
    torch.cuda.synchronize() if device == "cuda" else torch.mps.synchronize()
    end_time = time.perf_counter()
    print(f'Encoder execution time: {end_time - start_time:.2f} seconds')
        

print(f'Running decoder, test device: {device}')
## Run audio chunks to the decoder
wv_recons = []
with torch.no_grad():
    torch.cuda.synchronize() if device == "cuda" else torch.mps.synchronize()
    start_time = time.perf_counter()
    for i, latent in enumerate(latent_chunks):
        wv_recon = autoencoder.decode(latent)
        wv_recons.append(wv_recon)
    torch.cuda.synchronize() if device == "cuda" else torch.mps.synchronize()
    end_time = time.perf_counter()
    print(f'Decoder execution time: {end_time - start_time:.2f} seconds')
wv_recon = torch.cat(wv_recons, dim=-1)
print(f'reconstructed waveform shape: {wv_recon.shape}')

In [ ]:
"Original:"
display(Audio(wv.cpu().numpy()[0], rate=sr))
"Reconstructed:"
display(Audio(wv_recon.cpu().numpy()[0], rate=sr))